In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from itertools import permutations
from itertools import combinations 
from sklearn.metrics import accuracy_score
import seaborn as sns
import pandas as pd
import numpy as np 
import os

In [25]:
mat = np.array([[100,100,100],[100,250,180],[190,130,200]])#1
mat = np.array([[150,250,130],[100,120,100],[80,100,120]])#2
mat = np.array([[150,100,100],[200,250,180],[300,500,450]])#3
mat = np.array([[500,200,50],[400,10,10],[350,100,120]])#4
mat = mat/mat.max()
mat

array([[1.  , 0.4 , 0.1 ],
       [0.8 , 0.02, 0.02],
       [0.7 , 0.2 , 0.24]])

In [26]:
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(mat, square=True, cmap='Blues')
f.savefig('1.png')

In [2]:
#讀檔
file_name = 'forpy'
data_original = pd.read_csv(file_name + '.csv')

In [3]:
#創建放圖片資料夾
path = '5群結果'
if not os.path.isdir(os.path.abspath(path)):
    os.makedirs(os.path.abspath(path))

In [5]:
#要存結果的檔名
result_name = 'result'

In [6]:
#打亂順序，存成 file_name_shuffle.csv
data = shuffle(data_original)
data = data.reset_index(drop=True)
data.to_csv(file_name + '_shuffle.csv')

In [7]:
data

,0,0.1,1,0.2,0.3,0.4,0.5,0.6,1.1
0,1.0,0.0,0,0.666667,0.000000,0.53,0,0,5
1,0.0,0.0,1,0.000000,0.000000,0.00,0,0,4
2,0.0,0.5,0,0.666667,0.000000,0.46,0,1,1
3,0.0,0.0,0,0.000000,0.000000,0.37,0,1,3
4,0.0,0.0,1,0.000000,0.004082,0.01,0,0,3
...,...,...,...,...,...,...,...,...,...
2062,0.0,0.0,0,0.333333,0.000000,0.71,0,1,2
2063,0.0,0.0,1,0.000000,0.000000,0.00,0,0,2
2064,0.5,0.0,0,0.666667,0.000000,0.53,0,1,3
2065,0.0,0.0,1,0.000000,0.000000,0.39,0,1,3


# 列出所有排列組合

In [8]:
#列出所有排列組合，存在 All_set 內
col = data.columns
col = col[:-1]
lis = col 
all_set = []
for i in range(1, len(lis) + 1):
    all_set.append(list(combinations(lis, i)))
c = 0
All_set = []
for i in range(len(all_set)):
    c+=len(all_set[i])
    for a in range(len(all_set[i])):
        All_set.append(list(all_set[i][a]))
print('排列組合數量 =',c)

排列組合數量 = 255


In [9]:
#All_set

# 開始跑所有排列組合

In [10]:
def train_test_split(data, test_size):
    lenth = len(data)
    split = int(lenth*(1-test_size))
    train = data[:split]
    test = data[split:]
    return train, test

In [21]:
def run_model(path=path, data=data, All_set=All_set):
    '''
    可變參數：
    1. test_size : 測試集大小
    2. max_depth：隨機森林樹的層數
    '''
    no = []
    acc_score_list = []
    col_set = []
    for num in range(len(All_set)):
        i = All_set[num]
        x = data.loc[:,i]
        y = data.iloc[:,-1:]
        col_set.append(i)
        #切分測試訓練集
        test_size = 0.15
        x_train, x_test = train_test_split(x, test_size)
        y_train, y_test = train_test_split(y, test_size)
        #建立模型
        model = RandomForestClassifier(max_depth=1000, random_state=0)
        #訓練
        model.fit(x_train, y_train)
        #預測
        y_pred = model.predict(x_test)
        #畫混淆矩陣
        mat = confusion_matrix(y_test, y_pred)
        
        
        f, ax = plt.subplots(figsize=(9, 6))
        ax.set_title('No.{}'.format(str(num)))
        ax.set_xlabel('True label')
        mat = confusion_matrix(y_test, y_pred)
        ax = sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
        plt.ylabel('Predicted label')
        plt.xlabel('True label')
        
        f.savefig('{}/No.{}.png'.format(path,str(num)))
        
        
        
        acc_score = accuracy_score(y_test, y_pred, normalize=True)
        acc_score_list.append(acc_score)
        no.append(num)
    all_model = pd.DataFrame()
    all_model['No.'] = no
    all_model['set'] = col_set
    all_model['acc'] = acc_score_list
    return all_model

In [22]:
All_set_v2 = All_set[:10]
all_model = run_model(path=path, data=data, All_set=All_set_v2)

<ipython-input-21-c59b7e76e038>:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
<ipython-input-21-c59b7e76e038>:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
<ipython-input-21-c59b7e76e038>:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
<ipython-input-21-c59b7e76e038>:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
<ipython-input-21-c59b7e76e038>:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected

In [23]:
all_model = all_model.sort_values('acc',ascending=False)
all_model.to_csv(result_name+'.csv')